[View in Colaboratory](https://colab.research.google.com/github/dharmesh312/CVML-COURSE/blob/master/mnistLearning.ipynb)

In [0]:
import pandas as pd 
import numpy as np 
import tensorflow
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [26]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
# softmax without any convolution layer and basic softmax function 
def accuracy(predictions , labels):
  corretly_predicted = np.sum(np.argmax(predictions , 1) == np.argmax(labels,1))
  accur = (corretly_predicted / predictions.shape[0])*100
  return accur

validDataSet = mnist.validation.images 
validLabels = mnist.validation.labels
x = tf.placeholder("float", [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W) + b)
tf_valid_dataset = tf.constant(validDataSet)
valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset , W) + b)
y_ = tf.placeholder("float", [None,10])
# cross Entroy for using a value that we need to minimize
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
# sess = tf.InteractiveSession()
sess.run(init)
for i in range(100000):
  batch_xs, batch_ys = mnist.train.next_batch(50)
#   print (tf.RunOptions.FULL_TRACE)
  _ , l , predictions = sess.run([train_step , cross_entropy , y], feed_dict={x: batch_xs, y_: batch_ys} )
  
  if (i%50 == 0):
    print ("The batch step is : ",i ," with loss : " ,l , " And train accuracy is  " ,accuracy(predictions , batch_ys) , " Validation accurcy is: " ,accuracy(valid_prediction.eval(session = sess),validLabels) )

The batch step is :  0  with loss :  115.129265  And train accuracy is   12.0  Validation accurcy is:  22.7
The batch step is :  50  with loss :  29.205488  And train accuracy is   84.0  Validation accurcy is:  87.16000000000001
The batch step is :  100  with loss :  11.066402  And train accuracy is   96.0  Validation accurcy is:  88.24
The batch step is :  150  with loss :  33.86402  And train accuracy is   80.0  Validation accurcy is:  89.16
The batch step is :  200  with loss :  17.082449  And train accuracy is   88.0  Validation accurcy is:  90.16
The batch step is :  250  with loss :  15.904949  And train accuracy is   92.0  Validation accurcy is:  89.75999999999999
The batch step is :  300  with loss :  16.604778  And train accuracy is   90.0  Validation accurcy is:  90.03999999999999
The batch step is :  350  with loss :  14.454662  And train accuracy is   92.0  Validation accurcy is:  90.3
The batch step is :  400  with loss :  18.404358  And train accuracy is   96.0  Validatio

In [40]:
# prediction on the model for the test data
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))




0.9235
